In [1]:
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#initialize two empty lists
latlib_data = []
latlib_titles = []

In [3]:
#initialize an empty DataFrame
latlib_df = pd.DataFrame([])


In [4]:
class LatLibSpider(scrapy.Spider):
    name = 'latlib_spider'
    def start_requests(self):
        #This starts the process. Here the base url is included.
        
        urls = 'https://www.thelatinlibrary.com/'
        yield scrapy.Request( url = urls, callback = self.parse_front)
    def parse_front(self, response):
        #opening latinlibrary page
        #parser to get urls for each author's individual works
        
        author_links = response.xpath('.//table[2]/tr/td/a/@href').extract() 
        head_url = 'https://www.thelatinlibrary.com/'
        author_links = [head_url+l for l in author_links] 
        for link in author_links:
            yield response.follow(url = link, callback = self.parse_authors)
            
    def parse_authors(self, response):
        #navigate to each author page
        #This is to extract individual author links. 
        #This includes the xpath location of each author's html
        #then appends it onto the base url using list comprehension.
        
        author_page = response.xpath('//div[@class="work"]/table/tbody/tr/td/a/@href').extract() 
        for link in author_page:
            yield response.follow(url = link, callback = self.parse_works)

    def parse_works(self, response):
        #get the title of each work:
        #This parser is used to scrape the actual text and title from each work
        #as well as append that data to the empty lists at the top of the file.
        
        title_text = response.xpath('/html/head/title/text()').extract()
        #get the content of each work under the authors
        body_text = response.xpath('.//body/*/text()').extract() 

        #latlib_data = latlib_data.append(title_text)
        latlib_data.append(body_text) 
        latlib_titles.append(title_text) 
        
        


In [5]:
#This is where I actually run the Spider
process = CrawlerProcess()
process.crawl(LatLibSpider)
process.start()

2021-03-05 19:43:45 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-05 19:43:45 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 02:22:02) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform macOS-10.15.5-x86_64-i386-64bit
2021-03-05 19:43:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-05 19:43:45 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-05 19:43:45 [scrapy.extensions.telnet] INFO: Telnet Password: 7e86f700bebb2fcb
2021-03-05 19:43:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-03-05 19:43:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddle

2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/curtius/curtius9.shtml> (referer: https://www.thelatinlibrary.com/curtius.html)
2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/curtius/curtius7.shtml> (referer: https://www.thelatinlibrary.com/curtius.html)
2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/curtius/curtius6.shtml> (referer: https://www.thelatinlibrary.com/curtius.html)
2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/curtius/curtius5.shtml> (referer: https://www.thelatinlibrary.com/curtius.html)
2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/30.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justinian.

2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/28.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/26.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/25.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/24.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/23.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/29.shtml> (refere

2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius4.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius3.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius1.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/adbrutum.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/fratrem.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com

2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/cat.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/rabirio.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/sulla.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/murena.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/caecina.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-05 19:43:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/imp.shtml> (referer: https://www.the

...

2021-03-05 19:43:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann12.shtml> (referer: https://www.thelatinlibrary.com/tac.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann14.shtml> (referer: https://www.thelatinlibrary.com/tac.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann13.shtml> (referer: https://www.thelatinlibrary.com/tac.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann15.shtml> (referer: https://www.thelatinlibrary.com/tac.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann16.shtml> (referer: https://www.thelatinlibrary.com/tac.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/tacitus/tac.ann11.sht

2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/valmax3.html> (referer: https://www.thelatinlibrary.com/valmax.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/valmax1.html> (referer: https://www.thelatinlibrary.com/valmax.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/varro.ll10.html> (referer: https://www.thelatinlibrary.com/varro.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/valmax2.html> (referer: https://www.thelatinlibrary.com/valmax.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/varro.ll9.html> (referer: https://www.thelatinlibrary.com/varro.html)
2021-03-05 19:43:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/varro.ll8.html> (referer: https://www.thelatinlibrary.com/

2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/gellius/gellius3.shtml> (referer: https://www.thelatinlibrary.com/gellius.html)
2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sha/verus.shtml> (referer: https://www.thelatinlibrary.com/sha.html)
2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sha/marcant.shtml> (referer: https://www.thelatinlibrary.com/sha.html)
2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/gellius/gellius2.shtml> (referer: https://www.thelatinlibrary.com/gellius.html)
2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/gellius/gellius1.shtml> (referer: https://www.thelatinlibrary.com/gellius.html)
2021-03-05 19:43:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sha/ant.shtml> (referer: h

2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/14.html> (referer: https://www.thelatinlibrary.com/justin.html)
2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/13.html> (referer: https://www.thelatinlibrary.com/justin.html)
2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/11.html> (referer: https://www.thelatinlibrary.com/justin.html)
2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/10.html> (referer: https://www.thelatinlibrary.com/justin.html)
2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/9.html> (referer: https://www.thelatinlibrary.com/justin.html)
2021-03-05 19:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/justin/8.html> (referer: https://www.thelatinlibrary

2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.29.shtml> (referer: https://www.thelatinlibrary.com/liv.html)
2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.30.shtml> (referer: https://www.thelatinlibrary.com/liv.html)
2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.27.shtml> (referer: https://www.thelatinlibrary.com/liv.html)
2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.26.shtml> (referer: https://www.thelatinlibrary.com/liv.html)
2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.28.shtml> (referer: https://www.thelatinlibrary.com/liv.html)
2021-03-05 19:43:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/livy/liv.24.shtml> (referer: https://www.thelatinli

In [6]:
#here's an example of a text and its title!
print(latlib_titles[10])
print(latlib_data[10])


['\n\t\t\tCaesar: Bellum Gallicum V\n\t\t']
['C. IVLI CAESARIS COMMENTARIORVM DE BELLO GALLICO LIBER QVINTVS', '\n\n', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', '\n', '\n[', '] ', ' L. Domitio Ap. Claudio consulibus, discedens ab hibernis Caesar in Italiam, ut quotannis facere consuerat, legatis imperat quos legionibus praefecerat uti quam plurimas possent hieme naves aedificandas veteresque reficiendas curarent. Earum modum formamque demonstrat. ', ' Ad celeritatem onerandi subductionesque paulo facit humiliores quam quibus in nostro mari uti consuevimus, atque id eo magis, quod propter crebras commutationes aestuum minus magnos ibi f

In [7]:
#number of texts:
len(latlib_data)

673

In [8]:
#length of the first text:
len(latlib_data[0])

166

In [12]:
#This is where I converted the lists containing the title and text data into Series
#and added them into a DataFrame.
latlib_df['title'] = pd.Series(latlib_titles)
latlib_df['text'] = pd.Series(latlib_data)

In [10]:
latlib_df.sample(10)

,title,text
40,[\n\t\t\tP. VERGILI MARONIS AENEIDOS LIBER OCT...,[P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS\n\n...
456,[Tacitus: Histories IV\n\t\t],[P. CORNELI TACITI HISTORIARVM LIBER QVARTVS\n...
662,[\n\t\t\tLivy I\n\t\t],"[TITI LIVI AB VRBE CONDITA LIBER I\n, \n\n, \n..."
252,[\n\t\t\tOvid: Heroides IX\n\t\t],"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n..."
183,[\n\t\t\tde Grammaticis\n\t\t],"[C. SVETONI TRANQVILII DE GRAMMATICIS\n, \n\t,..."
361,[\n\t\t\tSeneca the Elder\n\t\t],[L. ANNAEI SENECAE MAIORIS EXCERPTA CONTROVERS...
237,[\n\t\t\tNepos: Praefatio\n\t\t],"[PRAEFATIO\n\n, \n, Non dubito fore plerosque..."
348,[\n\t\t\tQuintilian: Institutio Oratoria III\n...,[M. FABII QVINTILIANI INSTITVTIO ORATORIA LIB...
136,[\n\t\t\tCicero: Brutus\n\t\t\t\t\t],"[M. TVLLI CICERONIS BRVTVS\n\n, \n, \t, \t, ..."
254,[\n\t\t\tOvid: Heroides XI\n\t\t],"[P. OVIDI NASONIS EPISTVLAE HEROIDVM\n, \n, \n..."


In [13]:
#Here is the pickle:
latlib_df.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [14]:
#to make a set of sample data I'll take ten random files and pickle them:
samples = latlib_df.sample(10)

In [18]:
samples.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data_samples/ten_samples.pkl")